In [153]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [154]:
df=pd.read_csv(r"./final_cleaned_data.csv").iloc[:,1:]
df

,marketingStatusSimplifiedCd,hasImage,statusText,beds,baths,area,sgapt,has3DModel,hasAdditionalAttributions,latitude,longitude,city,state,price,homeType,rentZestimate
0,Non Owner Occupied,True,Sold,3.0,2.0,1224.0,Unknown Listed By,False,True,42.293304,-71.133570,Roslindale,MA,819500.0,SINGLE_FAMILY,3999.0
1,RecentChange,True,Sold,1.0,1.0,723.0,Unknown Listed By,False,True,42.344826,-71.074990,Boston,MA,880000.0,CONDO,3634.0
2,RecentChange,True,Sold,6.0,3.0,3396.0,Unknown Listed By,False,True,42.315850,-71.061130,Dorchester,MA,1250000.0,MULTI_FAMILY,6000.0
3,RecentChange,True,Sold,4.0,3.0,2400.0,Unknown Listed By,False,True,42.298183,-71.078870,Dorchester,MA,805000.0,MULTI_FAMILY,4964.0
4,RecentChange,True,Sold,9.0,3.0,4107.0,RecentChange,False,True,42.300050,-71.083694,Dorchester,MA,650000.0,MULTI_FAMILY,8970.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6201,RecentChange,True,Sold,5.0,4.0,2366.0,Unknown Listed By,False,True,38.963560,-77.016990,Washington,DC,1125000.0,SINGLE_FAMILY,4999.0
6202,RecentChange,True,Sold,3.0,3.0,1802.0,Unknown Listed By,False,True,38.952750,-77.009360,Washington,DC,465000.0,TOWNHOUSE,3449.0
6203,RecentChange,True,Sold,4.0,4.0,2875.0,RecentChange,False,True,38.928920,-76.992680,Washington,DC,1020000.0,SINGLE_FAMILY,4749.0
6204,Non Owner Occupied,False,Sold,4.0,2.5,1260.0,Unknown Listed By,False,False,38.894318,-76.981830,Washington,DC,550000.0,SINGLE_FAMILY,3994.0


- First we have to decrease number of categories in cities as there are too many unique cities
- All the cities that contribute less than 1% of total number of rows have been labeled as Others

In [155]:
n_cities=df["city"].nunique()
print(f"There are {n_cities} unique cities!")

There are 142 unique cities!


In [156]:
les_propor_cities=df["city"].value_counts()[(((df["city"].value_counts()/df["city"].shape[0])*100)<1)].index
df["city"]=df["city"].apply(lambda x:x if x not in les_propor_cities else "Others")
df["city"].value_counts()

city
Others           1077
Denver            486
Indianapolis      485
San Jose          471
Philadelphia      469
Charlotte         458
Phoenix           402
San Diego         368
Washington        358
Seattle           347
San Francisco     346
Nashville         324
Chicago           264
Los Angeles       183
Tampa              88
Boston             80
Name: count, dtype: int64

Clubbed the Pre-Foreclosure - RecentChange and Pre-Foreclosure categories from marketingStatusSimplifiedCd feature as both of them are very low in number                      

In [157]:
df["marketingStatusSimplifiedCd"].value_counts()

marketingStatusSimplifiedCd
RecentChange                      4933
Non Owner Occupied                1267
Pre-Foreclosure - RecentChange       5
Pre-Foreclosure                      1
Name: count, dtype: int64

In [158]:
x=df['marketingStatusSimplifiedCd'].value_counts()[df['marketingStatusSimplifiedCd'].value_counts()<100]
y=x.index
df["marketingStatusSimplifiedCd"]=df["marketingStatusSimplifiedCd"].apply(lambda x:x if x not in y else "Pre-Foreclosure")
df["marketingStatusSimplifiedCd"].value_counts()

marketingStatusSimplifiedCd
RecentChange          4933
Non Owner Occupied    1267
Pre-Foreclosure          6
Name: count, dtype: int64

Dropped latitude and longitude as they did not prove useful at all in EDA

In [159]:
df.drop(columns=['latitude','longitude'],axis=1,inplace=True)
print(df.shape)
df.columns

(6206, 14)


Index(['marketingStatusSimplifiedCd', 'hasImage', 'statusText', 'beds',
       'baths', 'area', 'sgapt', 'has3DModel', 'hasAdditionalAttributions',
       'city', 'state', 'price', 'homeType', 'rentZestimate'],
      dtype='object')

Imputation->Outlier Removal->Encoding->Scaling

# Null Value Imputation

In [160]:
df.isnull().sum()

marketingStatusSimplifiedCd      0
hasImage                         0
statusText                       0
beds                           186
baths                          134
area                           201
sgapt                            0
has3DModel                       0
hasAdditionalAttributions        0
city                             0
state                            0
price                            0
homeType                         0
rentZestimate                   62
dtype: int64

Imputing beds and baths with mode as it is discrete data type

In [161]:
for column in df.columns:
    if column=='beds'or column=='baths': 
        df[column].fillna(df[column].mode()[0], inplace=True)

df[["beds","baths"]]

,beds,baths
0,3.0,2.0
1,1.0,1.0
2,6.0,3.0
3,4.0,3.0
4,9.0,3.0
...,...,...
6201,5.0,4.0
6202,3.0,3.0
6203,4.0,4.0
6204,4.0,2.5


Area and restZestimate we are imputing using linear interpolation

In [162]:
numeric_columns = ['area','rentZestimate']  

for column in numeric_columns:
    df[column].interpolate(method='linear', inplace=True)

In [163]:
df.isnull().sum()

marketingStatusSimplifiedCd    0
hasImage                       0
statusText                     0
beds                           0
baths                          0
area                           0
sgapt                          0
has3DModel                     0
hasAdditionalAttributions      0
city                           0
state                          0
price                          0
homeType                       0
rentZestimate                  0
dtype: int64

# Encoding

Using One Hot Encoding for categorical features

In [164]:
import pandas as pd

categorical_cols = ['marketingStatusSimplifiedCd', 'hasImage','statusText', 'sgapt', 'has3DModel', 'hasAdditionalAttributions', 'city', 'state', 'homeType']

one_hot_encoded = pd.get_dummies(df[categorical_cols],drop_first=True)

df = pd.concat([one_hot_encoded, df[['beds', 'baths', 'area', 'rentZestimate', 'price']]], axis=1)

df

,hasImage,has3DModel,hasAdditionalAttributions,marketingStatusSimplifiedCd_Pre-Foreclosure,marketingStatusSimplifiedCd_RecentChange,statusText_Sold,sgapt_Unknown Listed By,city_Charlotte,city_Chicago,city_Denver,...,homeType_LOT,homeType_MANUFACTURED,homeType_MULTI_FAMILY,homeType_SINGLE_FAMILY,homeType_TOWNHOUSE,beds,baths,area,rentZestimate,price
0,True,False,True,False,False,True,True,False,False,False,...,False,False,False,True,False,3.0,2.0,1224.0,3999.0,819500.0
1,True,False,True,False,True,True,True,False,False,False,...,False,False,False,False,False,1.0,1.0,723.0,3634.0,880000.0
2,True,False,True,False,True,True,True,False,False,False,...,False,False,True,False,False,6.0,3.0,3396.0,6000.0,1250000.0
3,True,False,True,False,True,True,True,False,False,False,...,False,False,True,False,False,4.0,3.0,2400.0,4964.0,805000.0
4,True,False,True,False,True,True,False,False,False,False,...,False,False,True,False,False,9.0,3.0,4107.0,8970.0,650000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6201,True,False,True,False,True,True,True,False,False,False,...,False,False,False,True,False,5.0,4.0,2366.0,4999.0,1125000.0
6202,True,False,True,False,True,True,True,False,False,False,...,False,False,False,False,True,3.0,3.0,1802.0,3449.0,465000.0
6203,True,False,True,False,True,True,False,False,False,False,...,False,False,False,True,False,4.0,4.0,2875.0,4749.0,1020000.0
6204,False,False,False,False,False,True,True,False,False,False,...,False,False,False,True,False,4.0,2.5,1260.0,3994.0,550000.0


In [165]:
bool_cols=df.dtypes[df.dtypes=="bool"].index
for col in bool_cols:
    df[col]=df[col].astype(float)
df

,hasImage,has3DModel,hasAdditionalAttributions,marketingStatusSimplifiedCd_Pre-Foreclosure,marketingStatusSimplifiedCd_RecentChange,statusText_Sold,sgapt_Unknown Listed By,city_Charlotte,city_Chicago,city_Denver,...,homeType_LOT,homeType_MANUFACTURED,homeType_MULTI_FAMILY,homeType_SINGLE_FAMILY,homeType_TOWNHOUSE,beds,baths,area,rentZestimate,price
0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,3.0,2.0,1224.0,3999.0,819500.0
1,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,723.0,3634.0,880000.0
2,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,6.0,3.0,3396.0,6000.0,1250000.0
3,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,4.0,3.0,2400.0,4964.0,805000.0
4,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,9.0,3.0,4107.0,8970.0,650000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6201,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,5.0,4.0,2366.0,4999.0,1125000.0
6202,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,3.0,3.0,1802.0,3449.0,465000.0
6203,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,4.0,4.0,2875.0,4749.0,1020000.0
6204,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,4.0,2.5,1260.0,3994.0,550000.0


In [166]:
df.to_csv("./preprocessed.csv")